In [61]:
import folium
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from lxml import etree
import requests
from IPython.display import display, HTML
import re

In [62]:
projects= r'P3_GrantExport.csv'
state_data = pd.read_csv(projects, sep=';')
state_data.set_index(state_data.columns[0], inplace=True)
state_data = state_data[['University', 'Approved Amount']]
state_data.dropna(subset=['University'], inplace=True)
state_data = state_data[state_data['University'] != 'Nicht zuteilbar - NA'] #Nicht zuteilbar = NA University
state_data = state_data[state_data['University'] != 'NPO (Biblioth., Museen, Verwalt.) - NPO'] #NPO = is not university

In [105]:

def getData(project_id):
    url = 'http://p3.snf.ch'
    url_project = url + '/project-' + str(project_id)
    # We parse the response to analyze it 
    soup_project = BeautifulSoup(requests.request("GET", url_project).text, 'html.parser')
    id_institute = soup_project.find(id="ctl00_MainContent_urlInstitite").get('href')
    if id_institute is None:
        return None
    
    url_institute = url + id_institute
    
    toSoup = requests.request("GET", url_institute)
    soup_institute = BeautifulSoup(toSoup.text, 'html5lib')
    table = soup_institute.findAll('div', {'class': 'institute'})[0].findAll('td')[1].text
    p = re.compile("CH-\S{4}")
    match = p.search(table)
    if match is None:
        return None
    postal_code = match.group(0)
    return postal_code

In [106]:
def search_missing_cantons(UniCanton):
    for i in range(len(UniCanton)):
        if UniCanton['Canton'].iloc[i] == 'NaN':
            query = getData(temp.index.values[i])
            canton = searchCantonGeoNames(query)
            UniCanton['Canton'].iloc[i] = canton
    return UniCanton


In [ ]:
def searchCantonGeoNames(query1, query2=None): #returns in which canton the university is in
    url = 'http://api.geonames.org/searchJSON?country=CH&maxRows=1&username=jnls'
    payload = {'q': query1 }
    req = requests.get(url, params=payload)
    req_json = json.loads(req.text)
    if req_json['totalResultsCount'] > 0:
        return req_json['geonames'][0]['adminCode1']
    elif req_json['totalResultsCount'] == 0 and query2 is not None:
        payload = {'q': query2 }
        req = requests.get(url, params=payload)
        req_json = json.loads(req.text)
        if req_json['totalResultsCount'] > 0:
            return req_json['geonames'][0]['adminCode1']
        else:
            return "NaN"
def search_cantons(data):
    UniCanton = data['University'].drop_duplicates().to_frame()
    UniCanton['Canton'] = ''
    for i in range(len(UniCanton)):
        query = UniCanton['University'].iloc[i].split(" - ")
        if len(query) > 1:
            canton = searchCantonGeoNames(query[0], query[1])
        else:
            canton = searchCantonGeoNames(query[0])
        UniCanton['Canton'].iloc[i] = canton
    #data = data[data['Canton'] != 'NaN'] 
    return UniCanton

temp = search_cantons(state_data)
temp = search_missing_cantons(temp)
temp

In [ ]:
temp.set_index(temp.University, inplace=True, drop=True)
temp = temp.to_dict()['Canton']
state_data['Canton'] = state_data['University'].map(temp)
state_data.reset_index(drop=True, inplace=True)
grants = state_data.drop('University', 1)
grants['Approved Amount'] = grants['Approved Amount'].convert_objects(convert_numeric=True)
grants_sum = grants.groupby(grants.Canton).sum()
grant_canton = grants_sum['Approved Amount'].to_dict()


In [172]:
cantonMap = pd.DataFrame({'Canton':['JU', 'ZH', 'BE', 'LU', 'SG', 'NE', 'VD', 'VS', 'GE', 'TI', 'SO', 'UR', 'OW', 'NW', 'SZ', 'GL', 'ZG', 'BL', 'BS', 'FR', 'SH', 'AR', 'AI', 'GR', 'AG', 'TG']})
cantonMap['Grants'] = 0
cantonMap['Grants'] = cantonMap['Canton'].map(grant_per_canton)

ch_geo = r'ch-cantons.topojson.json'

map = folium.Map(location=[46.82244,8.22410], zoom_start=8)
map.choropleth(data=cantonMap, 
                columns=['Canton', 'Grants'], 
                geo_path=ch_geo,
                topojson='objects.cantons', 
                fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
                key_on='feature.id')
map.save('ch-cantons.html')

/Users/dmartr/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.
